# 构造微调训练数据集（Deepseek 版）
通过 LangChain 的 ChatDeepSeek 调用 Deepseek 模型，生成周易卦象的训练数据并保存为 CSV 文件。

In [1]:
# 步骤1：导入依赖库
import os
import csv
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [2]:
# 步骤2：初始化 Deepseek 客户端
# 从环境变量读取 API Key（需提前设置：export DEEPSEEK_API_KEY='你的密钥'）
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
if not deepseek_api_key:
    raise ValueError("请先设置环境变量 DEEPSEEK_API_KEY（参考：export DEEPSEEK_API_KEY='你的密钥'）")

# 初始化 ChatDeepSeek 模型（指定模型名为 deepseek-chat）
llm = ChatDeepSeek(
    model="deepseek-chat",  # Deepseek 对话模型
    api_key=deepseek_api_key
)

In [3]:
# 步骤3：定义系统提示词（与原 OpenAI 逻辑保持一致）
system_prompt = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：
师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：
content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""

In [4]:
# 步骤4：定义需要处理的周易卦象输入（可根据需求扩展）
卦象输入列表 = [
    # 蒙卦输入
    """蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。
蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。
《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。
《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。""",
    # 需卦输入
    """需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。
需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。 
需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。
《象》曰：云上于天，需;君子以饮食宴乐。
《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。
需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。""",
    # 讼卦输入
    """讼卦，上乾下坎，乾为天，坎为水。天西转，水东流，其行相反，象征争讼。
《序卦》曰：“有天地然后有万物，有万物然后有男女，有男女然后有夫妇，有夫妇然后有父子，有父子然后有君臣，有君臣然后有上下，有上下然后有礼仪有所措。夫妇之道，不可以不久也，故受之以恒。恒者，久也。物不可以终居其所，故受之以遁。遁者，退也。物不可以终遁，故受之以大壮。大壮者，进也。物不可以终进，故受之以晋。晋者，进也。进必有所伤，故受之以明夷。明夷者，伤也。伤于外者必反于家，故受之以家人。家道穷必乖，故受之以睽。睽者，乖也。乖必有难，故受之以蹇。蹇者，难也。难必有所解，故受之以解。解者，缓也。缓必有所失，故受之以损。损者，损也。损而不已必益，故受之以益。益而不已必决，故受之以夬。夬者，决也。决必有所遇，故受之以姤。姤者，遇也。物相遇而后聚，故受之以萃。萃者，聚也。聚而上者谓之升，故受之以升。升而不已必困，故受之以困。困乎上者必反下，故受之以井。井道不可不革，故受之以革。革物者莫若鼎，故受之以鼎。主器者莫若长子，故受之以震。震者，动也。物不可以终动，止之，故受之以艮。艮者，止也。物不可以终止，故受之以渐。渐者，进也。进必有所归，故受之以归妹。得其所归者必大，故受之以丰。丰者，大也。穷大者必失其居，故受之以旅。旅而无所容，故受之以巽。巽者，入也。入而后说之，故受之以兑。兑者，说也。说而后散之，故受之以涣。涣者，离也。物不可以终离，故受之以节。节而信之，故受之以中孚。有其信者必行之，故受之以小过。有过物者必济，故受之以既济。物不可穷也，故受之以未济。未济终焉。
《象》曰：天与水违行，讼；君子以作事谋始。""",
    # 师卦输入
    """师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。
《象》曰：地中有水，师；君子以容民畜众。""",
    # 比卦输入
    """比卦，由坤下坎上组成，坤为地，坎为水，象征地上有水，水附于地，地纳水，有比附、比和之意。比卦阐述的是君主亲附臣民，臣民亲附君主，相互比附、团结的道理。
《序卦》云：“师者，众也，众必有所比，故受之以比。比者，比也。”军队众多必然会有相互比附、团结之事，所以在师卦之后是比卦。
《象》曰：地上有水，比；先王以建万国，亲诸侯。""",
    # 小畜卦输入
    """小畜卦，乾下巽上，乾为天，巽为风，风行天上，象征小有蓄积。此卦表示蓄积微薄，但只要坚守正道，隐忍待时，亦可亨通，利于涉大川，有密云不雨之象。
《序卦》言：“比必有所畜，故受之以小畜。”事物相互比附之后，会有一定的蓄积，所以比卦之后是小畜卦。
《象》曰：风行天上，小畜；君子以懿文德。""",
    # 履卦输入
    """履卦，兑下乾上，兑为泽，乾为天，泽上有天，象征履行、践履。履卦强调循礼而行，谨慎行事，以“履虎尾，不咥人”为喻，说明行事需小心遵循规律，方能亨通。
《序卦》曰：“物畜然后有礼，故受之以履。”事物有了蓄积之后，就需要遵循礼仪规范，所以小畜卦之后是履卦。
《象》曰：上天下泽，履；君子以辨上下，定民志。""",
    # 泰卦输入
    """泰卦，坤下乾上，坤为地，乾为天，天地交合，阴阳交通，象征通泰、太平。泰卦表示事物处于顺利、畅达的状态，此时君子宜有所作为，利于君子道长，小人道消。
《序卦》云：“履而泰，然后安，故受之以泰。泰者，通也。”履行遵循礼仪规范后，会进入通泰的状态，所以履卦之后是泰卦。
《象》曰：天地交，泰；后以财成天地之道，辅相天地之宜，以左右民。""",
    # 否卦输入
    """否卦，乾下坤上，乾为天，坤为地，天地不交，阴阳隔绝，象征闭塞、不通。否卦表示事物处于困境、阻滞的状态，此时需坚守正道，等待时机，否极泰来。
《序卦》言：“泰者，通也。物不可以终通，故受之以否。”事物不可能永远通泰，所以泰卦之后是否卦。
《象》曰：天地不交，否；君子以俭德辟难，不可荣以禄。""",
    # 同人卦输入
    """同人卦，离下乾上，离为火，乾为天，天在上，火性炎上，同人于野，象征和同于人、团结一心。此卦强调与人和谐相处，同心同德，利于涉大川，亨通。
《序卦》曰：“物不可以终否，故受之以同人。”事物不能永远处于闭塞不通的状态，所以否卦之后是同人卦。
《象》曰：天与火，同人；君子以类族辨物。""",
    # 大有卦输入
    """大有卦，乾下离上，乾为天，离为火，火在天上，明亮普照，象征大有收获、富有。大有卦表示富有之时要秉持正道，与人和同，这样才能长久亨通。
《序卦》云：“与人同者，物必归焉，故受之以大有。”能与人和同的人，万物都会归附，所以同人卦之后是大有卦。
《象》曰：火在天上，大有；君子以遏恶扬善，顺天休命。""",
    # 谦卦输入
    """谦卦，坤下艮上，坤为地，艮为山，山在地下，象征谦虚。谦卦强调为人处世要谦虚谨慎，有谦德者亨通，利于涉大川，是吉祥之卦。
《序卦》言：“有大者不可以盈，故受之以谦。”有大的成就、富有之后不可以盈满自傲，所以大有卦之后是谦卦。
《象》曰：地中有山，谦；君子以裒多益寡，称物平施。""",
    # 豫卦输入
    """豫卦，坤下震上，坤为地，震为雷，雷出地奋，象征安乐、豫悦。豫卦说明在安乐之时要居安思危，不可沉迷享乐，遵循正道则亨通，利于建侯行师。
《序卦》曰：“谦者，谦也，谦必有所乐，故受之以豫。”有谦虚之德的人必定会有安乐之事，所以谦卦之后是豫卦。
《象》曰：雷出地奋，豫；先王以作乐崇德，殷荐之上帝，以配祖考。""",
    # 随卦输入
    """随卦，震下兑上，震为雷，兑为泽，泽中有雷，象征随从、相随。随卦强调随从正道，随从善者，这样才能亨通，利于君子贞。
《序卦》云：“豫必有随，故受之以随。”安乐豫悦之时必定会有随从之事，所以豫卦之后是随卦。
《象》曰：泽中有雷，随；君子以向晦入宴息。""",
    # 蛊卦输入
    """蛊卦，巽下艮上，巽为风，艮为山，山下有风，象征整治、匡正。蛊卦表示当事物出现弊端、混乱时，需要加以整治，秉持正道则可亨通，利于涉大川。
《序卦》言：“以喜随人者必有事，故受之以蛊。蛊者，事也。” 以喜悦之心随从别人，必定会有整治之事（处理事务），所以随卦之后是蛊卦。
《象》曰：山下有风，蛊；君子以振民育德。""",
    # 临卦输入
    """临卦，坤下兑上，坤为地，兑为泽，泽上有地，象征监临、统御。临卦表示上位者以正道监临、管理下位者，利于观民、教民，亨通，利于贞。
《序卦》曰：“有事而后可大，故受之以临。临者，大也。” 有整治事务的基础后，事业可以壮大，所以蛊卦之后是临卦。
《象》曰：泽上有地，临；君子以教思无穷，容保民无疆。""",
    # 观卦输入
    """观卦，坤下巽上，坤为地，巽为风，风行地上，象征观察、审视。观卦强调以正道观察事物，可洞察事理，利于祭祀、观民，有观仰之象。
《序卦》云：“临而可观，故受之以观。” 有监临统御的基础后可以进行观察审视，所以临卦之后是观卦。
《象》曰：风行地上，观；先王以省方，观民设教。""",
    # 噬嗑卦输入
    """噬嗑卦，震下离上，震为雷，离为火，雷电交加，象征咬合、刑狱。噬嗑卦表示处理事物需果断决策，就像咬合硬物一样，利于断狱、涉大川，亨通。
《序卦》言：“可观而后有所合，故受之以噬嗑。” 观察审视之后会有事物聚合（需要处理），所以观卦之后是噬嗑卦。
《象》曰：雷电，噬嗑；先王以明罚敕法。""",
    # 贲卦输入
    """贲卦，离下艮上，离为火，艮为山，山下有火，象征文饰、美化。贲卦强调文饰需适度，以质为本，文质兼备则亨通，有修饰之象。
《序卦》曰：“嗑者，合也，物合而后有礼，故受之以贲。贲者，饰也。” 事物咬合聚合之后需要文饰礼仪，所以噬嗑卦之后是贲卦。
《象》曰：山下有火，贲；君子以明庶政，无敢折狱。""",
    # 剥卦输入
    """剥卦，坤下艮上，坤为地，艮为山，山附于地，象征剥落、消蚀。剥卦表示事物处于逐渐衰败、被剥落的状态，此时需坚守正道，等待时机，以防不利。
《序卦》云：“贲者，饰也，致饰然后亨则尽矣，故受之以剥。剥者，剥也。” 过度文饰之后亨通的极致就会走向剥落，所以贲卦之后是剥卦。
《象》曰：山附于地，剥；上以厚下安宅。""",
    # 复卦输入
    """复卦，震下坤上，震为雷，坤为地，雷在地中，象征回复、归来。复卦表示事物衰败到极致后开始回复、新生，利于坚守正道，亨通，有一阳来复之象。
《序卦》言：“物不可以终尽剥，穷上反下，故受之以复。” 事物不能永远处于剥落消蚀的状态，到了极致就会向相反方向转化，所以剥卦之后是复卦。
《象》曰：雷在地中，复；先王以至日闭关，商旅不行，后不省方。""",
    # 无妄卦输入
    """无妄卦，震下乾上，震为雷，乾为天，天下雷行，象征不妄为、守正。无妄卦表示行事需遵循正道，不妄动、不妄求，亨通，利于守正，利于涉大川。
《序卦》曰：“复则不妄矣，故受之以无妄"""
]

In [5]:
# 步骤5：调用 Deepseek 生成训练数据
训练数据列表 = []

for 卦象输入 in 卦象输入列表:
    # 构造消息列表（系统提示 + 用户输入）
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=卦象输入)
    ]
    
    # 调用模型生成结果
    response = llm.invoke(messages)
    generated_content = response.content
    
    # 解析生成结果（按原格式提取 content 和 summary）
    # 注：实际使用时可根据生成结果优化解析逻辑
    lines = generated_content.strip().split('\n')
    content = lines[0].split(':"')[1].replace('"', '').strip()  # 提取 content 值
    summary = '\n'.join(lines[1:]).split(':"')[1].replace('"', '').strip()  # 提取 summary 值
    
    # 添加到训练数据列表
    训练数据列表.append({
        'content': content,
        'summary': summary
    })
    
    print(f"已处理：{content}")  # 打印进度

已处理：蒙卦
已处理：需卦
已处理：讼卦
已处理：师卦
已处理：比卦
已处理：小畜卦
已处理：履卦
已处理：泰卦
已处理：否卦
已处理：同人卦
已处理：大有卦
已处理：谦卦
已处理：豫卦
已处理：随卦
已处理：蛊卦
已处理：临卦
已处理：观卦
已处理：噬嗑卦
已处理：贲卦
已处理：剥卦
已处理：复卦
已处理：无妄卦


In [6]:
# 步骤6：保存为 CSV 文件
output_file = 'data/zhouyi_train_dataset_lizhe.csv'

with open(output_file, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['content', 'summary'])
    writer.writeheader()  # 写入表头
    writer.writerows(训练数据列表)  # 写入数据

print(f"数据集已保存至：{output_file}")

数据集已保存至：data/zhouyi_train_dataset_lizhe.csv
